In [205]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from datetime import datetime
import wandb
import argparse
from torch import nn, optim

# TitanicDataset

In [206]:
class TitanicDataset(Dataset):
    def __init__(self, X, y):
        '''데이터셋 초기화'''
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)
        
    def __len__(self):
        '''데이터셋 전체 길이를 반환'''
        return len(self.X)
    
    def __getitem__(self, idx):
        '''데이터셋의 특정 인덱스(feature) 반환'''
        feature = self.X[idx]
        target = self.y[idx]    # 타겟 데이터
        return {'input':feature, 'target': target}
    
    def __str__(self):
        '''데이터셋의 정보 알려주는 함수'''
        str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(len(self.X), self.X.shape, self.y.shape
        )
        return str

# TitanicTestDataset

In [207]:
class TitanicTestDataset(Dataset):
  def __init__(self, X):
    self.X = torch.FloatTensor(X)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    return {'input': feature}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str

# get_preprocessed_dataset

In [208]:
def get_preprocessed_dataset():
    # 파일의 절대 경로 설정
    CURRENT_FILE_PATH = os.path.dirname(os.path.abspath("__file__"))
    
    # train.csv 파일 경로 설정
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    # test.csv 파일 경로 설정
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")
    
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)
    
    # train.csv : PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Enbarked
    # test.csv : PassengerId, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Enbarked
    # 데이터프레임을 연결할 때 컬럼의 순서나 이름이 일치하지 않아도, 해당 컬럼 이름이 존재하는 경우 데이터를 합침
    all_df = pd.concat([train_df, test_df], sort=False)

    all_df = get_preprocessed_dataset_1(all_df)

    all_df = get_preprocessed_dataset_2(all_df)

    all_df = get_preprocessed_dataset_3(all_df)

    all_df = get_preprocessed_dataset_4(all_df)

    all_df = get_preprocessed_dataset_5(all_df)

    all_df = get_preprocessed_dataset_6(all_df)

    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset)
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

# get_preprocessed_dataset_1

In [209]:
def get_preprocessed_dataset_1(all_df):
    '''Pclass별 Fare 평귱값을 사용하여 Fare 결측치 메우기'''
    # Pcalss 와 Fare을 선택하여 데이터 추출 후 Pclass로 그룹화
    # 그 후 나온 평균 값을 Fare_mean에 저장
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    # Fare_mean 열의 이름을 Pclass 와 Fare_mean으로 변경
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    # Fare_mean 데이터프레임을 Pclass 열을 기준으로 left join 하여 병합
    # Pclass 열을 기준으로 Fare의 결측치를 Fare_mean 값으로 채울 수 있음
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]
    
    return all_df

# get_preprocessed_dataset_2

In [210]:
def get_preprocessed_dataset_2(all_df):
    '''name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침'''
    # , 또는 .을 기준으로 이름을 세 부분으로 분리 ex) Mr.Owen Karris -> Mr 과 Owen 과 Karris로 분리
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    # "family_name," "honorific," "name" 순서대로 이름이 분리
    name_df.columns = ["family_name", "honorific", "name"]
    # 각 이름들을 공백을 제거한 형태로 저장
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    # name_df을 열 방향(axis=1)으로 합침
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

# get_preprocessed_dataset_3

In [ ]:
def get_preprocessed_dataset_3(all_df):
    ''' hoorific별 Age 평균값을 사용하여 Age 결측치 메우기'''
    # honorific열 과 Age열을 honorific으로 묶고 중앙을 계산하여 Age값에서 반올림한다.
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    # honorific_age_mean 데이터프레임을 honorific 열을 기준으로 left join 하여 병합
    # honorific 열을 기준으로 Age 열의 결측치를 honorific_age_mean 값으로 채울 수 있음
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    # 필요 없어진 honorific_age_mean 열을 삭제
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df

# get_preprocessed_dataset_4

In [212]:
def get_preprocessed_dataset_4(all_df):
    # Parch열 과 SibSp열을 합쳐, 가족수(family_num) 컬럼 새롭게 추가
    # Parch : 부모/자녀 수
    # SibSp : 형제/자매/배우자 수
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]
    
    # 혼자탑습(alone) 컬럼 새롭게 추가
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)
    
    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)
    
    return all_df

# get_preprocessed_dataset_5

In [213]:
def get_preprocessed_dataset_5(all_df):
    ''' honorific 값 개수 줄이기'''
    # honorific열에서 Mr, Miss, Mrs, Master이 아닌 경우 해당 값을 other로 변경
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    # fillna 함수를 사용하여 탑승지(Embarked)열의 결측치를 missing으로 변경
    # inplace=False가 기본, True로 설정하면 데이터프레임 자체를 변경
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df

# get_preprocessed_dataset_6

In [214]:
def get_preprocessed_dataset_6(all_df):
    '''카테고리 변수(범주형, 문자열)를 LabelEncoder를 사용하여 수치값(정수)으로 변경하기'''
    # all_df에서 데이터 타입이 문자열(object)인 모든 열을 찾아서 category_features에 저장
    category_features = all_df.columns[all_df.dtypes == "object"]
    # LabelEncoder는 범주형 데이터를 정수로 변환하는 데 사용되는 클래스
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
            # fit 메서드를 사용하여 해당 열의 고유한 범주값들을 학습
            le = le.fit(all_df[category_feature])
            # transform 메서드를 사용하여 해당 열의 모든 값을 해당 범주의 정수로 변환
            all_df[category_feature] = le.transform(all_df[category_feature])
            
    return all_df

# 신경망 정의

In [215]:
from torch import nn
class MyModel(nn.Module):
    '''신경망 정의 / MyModel클래스는 nn.Module클래스를 상속받아 pytorch의 모델 기능을 활용할 수 있도록 함'''
    # input : feature의 수
    # n_output : 출력되어 나오는 결과
    def __init__(self, n_input, n_output):
        # 부모 클래스의 nn.Module의 생성자를 호출
        super().__init__()
        
        # 시퀀셜(Sequential) 모델을 사용, 이를 통해 여러 계층을 순차적으로 쌓을 수 있음
        # 시퀀셜 모델은 각 계층을 순차적으로 연결
        self.model = nn.Sequential(
            # n_input에서 30개의 뉴런을 가진 완전 연결 레이어를 생성
            nn.Linear(n_input, 30),
            nn.ReLU(),
            nn.Linear(30, 30),
            nn.ReLU(),
            nn.Linear(30, n_output),
        
            # 30개의 뉴런에서 출력 클래스 수 n_output으로 연결
            #nn.Linear(30, n_output),
        )
        
    # foward
    #  모델에 입력 데이터 x를 전달하여 출력을 반환
    def forward(self, x):
        # 입력 데이터 x를 모델의 시퀀셜 레이어에 전달하여 feed fowarding 수행
        x = self.model(x)
        return x

# test

In [217]:
def test(test_data_loader):
    '''모델을 테스트 하는 부분'''
    print("[TEST]")
    # iter를 사용하여 감싸준 다음 next를 사용하여 첫번째 미니배치를 배치로 넘김
    batch = next(iter(test_data_loader))
    print("{0}".format(batch['input'].shape))
    my_model = MyModel(n_input=11, n_output=2)
    output_batch = my_model(batch['input'])
    # 모델의 출력 중에서 각 샘플에 대한 가장 높은 값을 가진 클래스의 인덱스를 선택하여 예측
    prediction_batch = torch.argmax(output_batch, dim=1)
    # 인덱스를 892부터 시작하도록 설정, 첫번째 테스트 데이터의 인덱스 번째
    for idx, prediction in enumerate(prediction_batch, start=892):
        print(idx, prediction.item())

# main문

In [204]:
if __name__ == "__main__":
    # wandb 초기화
    wandb.init(project="titanic", name="ReLU")
    
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

    print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
        len(train_dataset), len(validation_dataset), len(test_dataset)
    ))
    print("#" * 50, 1)

    for idx, sample in enumerate(train_dataset):
        print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

    print("#" * 50, 2)

    train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    # 모델 정의
    my_model = MyModel(n_input=11, n_output=2)

    # 손실 함수 정의 (예: 크로스 엔트로피)
    criterion = torch.nn.CrossEntropyLoss()

    # 옵티마이저 설정 (예: 확률적 경사 하강법 - SGD)
    optimizer = torch.optim.SGD(my_model.parameters(), lr=0.01)

    # 학습 루프
    num_epochs = 1500  # 에폭 수를 설정
    for epoch in range(num_epochs):
        my_model.train()  # 모델을 학습 모드로 설정
        running_loss = 0.0

        for batch in train_data_loader:
            inputs = batch['input']
            labels = batch['target']

            optimizer.zero_grad()
            outputs = my_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        # Training loss를 wandb에 기록
        wandb.log({"Training Loss": running_loss / len(train_data_loader)})

        print(f"Epoch {epoch + 1}, Training Loss: {running_loss / len(train_data_loader)}")

        # 검증 부분
        my_model.eval()  # 모델을 평가 모드로 설정
        validation_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for batch in validation_data_loader:
                inputs = batch['input']
                labels = batch['target']

                outputs = my_model(inputs)
                loss = criterion(outputs, labels)
                validation_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        
        # Validation loss와 accuracy를 wandb에 기록
        wandb.log({"Validation Loss": validation_loss / len(validation_data_loader), "Validation Accuracy": accuracy})
        
        print(f"Epoch {epoch + 1}, Validation Loss: {validation_loss / len(validation_data_loader)}, Validation Accuracy: {accuracy:.2f}%")

    print("#" * 50, 3)

    # 테스트 함수 호출
    test(test_data_loader)
    
    wandb.finish()


################################################## 3
[TEST]
torch.Size([418, 11])
892 0
893 0
894 0
895 0
896 0
897 0
898 0
899 0
900 0
901 0
902 0
903 0
904 0
905 0
906 0
907 0
908 0
909 0
910 0
911 0
912 0
913 0
914 0
915 0
916 0
917 0
918 0
919 0
920 0
921 0
922 0
923 0
924 0
925 0
926 0
927 0
928 0
929 0
930 0
931 0
932 0
933 0
934 0
935 0
936 0
937 0
938 0
939 0
940 0
941 0
942 0
943 0
944 0
945 0
946 0
947 0
948 0
949 0
950 0
951 0
952 0
953 0
954 0
955 0
956 0
957 0
958 0
959 0
960 0
961 0
962 0
963 0
964 0
965 0
966 0
967 0
968 0
969 0
970 0
971 0
972 0
973 0
974 0
975 0
976 0
977 0
978 0
979 0
980 0
981 0
982 0
983 0
984 0
985 0
986 0
987 0
988 0
989 0
990 0
991 0
992 0
993 0
994 0
995 0
996 0
997 0
998 0
999 0
1000 0
1001 0
1002 0
1003 0
1004 0
1005 0
1006 0
1007 0
1008 0
1009 0
1010 0
1011 0
1012 0
1013 0
1014 0
1015 0
1016 0
1017 0
1018 0
1019 0
1020 0
1021 0
1022 0
1023 0
1024 0
1025 0
1026 0
1027 0
1028 0
1029 0
1030 0
1031 0
1032 0
1033 0
1034 0
1035 0
1036 0
1037 0
1038

Training Loss,█▇▇▇▆▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▁
Validation Accuracy,▂▃▁▃▃▂▄▃▅▅▅▅▅▁▆▄▅▇▆▄▇▇▇▆▄▇▆█▄▅▃▅▃█▅▂▆▆▄▇
Validation Loss,▄▅▅▅▄▄▄▇▂▄▃▂▃█▂▅▅▄▂▄▂▁▃▂▄▂▂▁▃▃█▃▅▁▃▇▂▂▅▄
Training Loss,0.39677
Validation Accuracy,77.52809
Validation Loss,0.55895
